In [137]:
import sys
from collections import Counter, OrderedDict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from timeit import timeit
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

#from inverted_index_colab import *
from binary_inverted_index_colab import *

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
# These will already be installed in the testing environment so disregard the 
# amount of time (~1 minute) it takes to install. 
!pip install -q pyspark
!pip install -U -q PyDrive
!apt-get update
!apt install openjdk-8-jdk-headless -qq 
!pip install -q graphframes
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
graphframes_jar = 'https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar'
spark_jars = '/usr/local/lib/python3.7/dist-packages/pyspark/jars'
!wget -N -P $spark_jars $graphframes_jar

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [73.9 kB]
Hit:12 http://ppa.launchpad.net/cran/

In [8]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from graphframes import *

In [9]:
# Initializing spark context
# create a spark context and session
conf = SparkConf().set("spark.ui.port", "4050")
sc = pyspark.SparkContext(conf=conf)
sc.addPyFile(str(Path(spark_jars) / Path(graphframes_jar).name))
spark = SparkSession.builder.getOrCreate()

In [10]:
spark

In [11]:
# Authenticate your user
# The authentication should be done with the email connected to your GCP account
from google.colab import auth
auth.authenticate_user()

In [12]:
# Copy one wikidumps files 
import os
from pathlib import Path
from google.colab import auth

project_id = 'core-period-321814'
!gcloud config set project {project_id}

data_bucket_name = 'wikidata_preprocessed'
try:
    if os.environ["wikidata_preprocessed"] is not None:
        pass  
except:
      !mkdir wikidumps
      !gsutil cp gs://{data_bucket_name}/multistream1_preprocessed.parquet "wikidumps/" 

Updated property [core/project].


To take a quick anonymous survey, run:
  $ gcloud survey

Copying gs://wikidata_preprocessed/multistream1_preprocessed.parquet...
- [1 files][316.7 MiB/316.7 MiB]                                                
Operation completed over 1 objects/316.7 MiB.                                    


In [13]:
from pathlib import Path 
import os

try:
    if os.environ["wikidata_preprocessed"] is not None:
      path = os.environ["wikidata_preprocessed"]+"/wikidumps/*"
except:
      path = "wikidumps/*"

parquetFile = spark.read.parquet(path)
parquetFile.show()

+---+--------------------+--------------------+--------------------+
| id|               title|                text|         anchor_text|
+---+--------------------+--------------------+--------------------+
| 12|           Anarchism|'''Anarchism''' i...|[{23040, politica...|
| 25|              Autism|'''Autism''' is a...|[{492271, Clinica...|
| 39|              Albedo|thumb|upright=1.3...|[{679294, diffuse...|
|290|                   A|'''A''', or '''a'...|[{290, See below}...|
|303|             Alabama|'''Alabama''' () ...|[{351590, Yellowh...|
|305|            Achilles|thumb|260px|Ancie...|[{1076007, potter...|
|307|     Abraham Lincoln|'''Abraham Lincol...|[{1827174, Alexan...|
|308|           Aristotle|'''Aristotle''' (...|[{1389981, bust},...|
|309|An American in Paris|'''''An American ...|[{13066, George G...|
|316|Academy Award for...|The '''Academy Aw...|[{39842, Academy ...|
|324|      Academy Awards|The '''Academy Aw...|[{649481, film in...|
|330|             Actrius|'''''Act

In [80]:
# take the 'title' and 'id' or the first 1000 rows and create an RDD from it
doc_title_pairs = parquetFile.limit(1000).select("title", "id").rdd

# take the 'anchor' and 'id' or the first 1000 rows and create an RDD from it
doc_anchor_pairs = parquetFile.limit(1000).select("anchor_text", "id").rdd

In [81]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ['category', 'references', 'also', 'links', 'extenal', 'see', 'thumb']
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

all_stopwords = english_stopwords.union(corpus_stopwords)

In [82]:
def word_to_docid(title, id):
  ''' 
  -----------
    title: str
      Title of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, doc_id) pairs 
      for example: [("Anarchism", 12), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(title.lower())]

  tuple_list = [] # creating the list
  for term in np.unique(tokens): # we only need a unique occurence of each term since the search on it will be binary
    tuple_list.append((term, id)) # adding to the list

  return tuple_list

In [83]:
def g(x):
  print(x)

In [85]:
word_docid = doc_title_pairs.flatMap(lambda x: word_to_docid(x[0], x[1]))
print(word_docid.collect())

[('anarchism', 12), ('autism', 25), ('albedo', 39), ('alabama', 303), ('achilles', 305), ('abraham', 307), ('lincoln', 307), ('aristotle', 308), ('american', 309), ('paris', 309), ('academy', 316), ('award', 316), ('best', 316), ('design', 316), ('for', 316), ('production', 316), ('academy', 324), ('awards', 324), ('actrius', 330), ('animalia', 332), ('book', 332), ('atomic', 334), ('international', 334), ('time', 334), ('altruism', 336), ('ayn', 339), ('rand', 339), ('alain', 340), ('connes', 340), ('allan', 344), ('dwan', 344), ('algeria', 358), ('atlas', 359), ('characters', 359), ('list', 359), ('shrugged', 359), ('anthropology', 569), ('agricultural', 572), ('science', 572), ('alchemy', 573), ('alien', 579), ('astronomer', 580), ('ascii', 586), ('austin', 590), ('disambiguation', 590), ('animation', 593), ('apollo', 594), ('agassi', 595), ('andre', 595), ('austroasiatic', 597), ('languages', 597), ('afroasiatic', 599), ('languages', 599), ('andorra', 600), ('arithmetic', 612), ('m

In [70]:
def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples 
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  # YOUR CODE HERE
  return sorted(unsorted_pl)

In [87]:
postings = word_docid.groupByKey().mapValues(reduce_word_counts)
print(postings.collect())

[('anarchism', [12]), ('autism', [25]), ('albedo', [39]), ('alabama', [303, 1285, 1286, 1484]), ('achilles', [305, 2421]), ('abraham', [307, 1436]), ('lincoln', [307]), ('aristotle', [308]), ('american', [309, 615, 659, 771, 863, 869, 966, 1110, 1111, 1112, 1201, 1241, 1488, 1558, 1890, 1902, 1950, 1973, 2011, 2175, 2357, 2386]), ('paris', [309]), ('academy', [316, 324]), ('award', [316]), ('best', [316]), ('design', [316]), ('for', [316, 1063]), ('production', [316]), ('awards', [324]), ('actrius', [330]), ('animalia', [332]), ('book', [332, 1726]), ('atomic', [334, 673, 1194, 1200, 1206, 2014]), ('international', [334]), ('time', [334, 1638]), ('altruism', [336]), ('ayn', [339]), ('rand', [339]), ('alain', [340, 1485]), ('connes', [340]), ('allan', [344]), ('dwan', [344]), ('algeria', [358]), ('atlas', [359, 780]), ('characters', [359]), ('list', [359, 728, 1286, 2142]), ('shrugged', [359]), ('anthropology', [569]), ('agricultural', [572]), ('science', [572]), ('alchemy', [573]), ('a

In [140]:
NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def partition_postings_and_write(postings):
  ''' A function that partitions the posting lists into buckets, writes out 
  all posting lists in a bucket to disk, and returns the posting locations for 
  each bucket. Partitioning should be done through the use of `token2bucket` 
  above. Writing to disk should use the function  `write_a_posting_list`, a 
  static method implemented in inverted_index_colab.py under the InvertedIndex 
  class. 
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and 
      offsets its posting list was written to. See `write_a_posting_list` for 
      more details.
  '''
 
  '''
  taking a posting of the form (w, posting_list)
  and returning (bucket_id, (w, posting_list))
  '''

  def mapping_bucket_id(term, posting):
    return (token2bucket_id(term), posting)

  def write_id(id_mapped_postings):
    return InvertedIndex.write_a_posting_list(id_mapped_postings)

  locations = postings.map(lambda x: mapping_bucket_id(x[0], x)).groupByKey().map(lambda x: write_id(x))

  return locations
  # def mapping_bucket_id(posting):
  #   return (token2bucket_id(posting[0]), posting)
  
  # def write_id(id_mapped_postings):
  #   return InvertedIndex.write_a_posting_list(id_mapped_postings)

  # locations = postings.flatMap(lambda x: )
  # locations = postings.map(mapping_bucket_id).groupByKey().map(write_id)
  
  #return locations

In [141]:
posting_locs_list = partition_postings_and_write(postings).collect()
print(posting_locs_list)

[defaultdict(<class 'list'>, {'anarchism': [('81_000.bin', 0)], 'agricultural': [('81_000.bin', 4)], 'schopenhauer': [('81_000.bin', 8)], 'politician': [('81_000.bin', 12)], 'disabilities': [('81_000.bin', 16)], 'abydos': [('81_000.bin', 20)], 'amyntas': [('81_000.bin', 32)], 'titus': [('81_000.bin', 40)], 'mahagonny': [('81_000.bin', 44)]}), defaultdict(<class 'list'>, {'autism': [('59_000.bin', 0)], 'abel': [('59_000.bin', 4)], 'angst': [('59_000.bin', 12)], 'casa': [('59_000.bin', 16)], 'arachnophobia': [('59_000.bin', 24)], 'serbia': [('59_000.bin', 28)], 'anderson': [('59_000.bin', 32)], 'andromeda': [('59_000.bin', 36)], 'achaeans': [('59_000.bin', 40)], 'angles': [('59_000.bin', 44)]}), defaultdict(<class 'list'>, {'albedo': [('2_000.bin', 0)], 'angolan': [('2_000.bin', 4)], 'aluminium': [('2_000.bin', 8)], 'antiparticle': [('2_000.bin', 12)], 'acacia': [('2_000.bin', 16)], 'voting': [('2_000.bin', 20)], 'acorn': [('2_000.bin', 24)], 'fox': [('2_000.bin', 28)], 'hill': [('2_000.

In [147]:
def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  # YOUR CODE HERE
  def to_df(posting):
    return (posting[0], len(posting[1]))

  return postings.map(to_df)

In [149]:
# global statistics
w2df = calculate_df(postings)
w2df_dict = w2df.collectAsMap()

In [142]:
# merge the posting locations into a single dict and run more tests (5 points)
super_posting_locs = defaultdict(list)
for posting_loc in posting_locs_list:
  for k, v in posting_loc.items():
    super_posting_locs[k].extend(v)

In [150]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
# write the global stats out
inverted.write_index('.', 'index')

In [151]:
TUPLE_SIZE = 4       
TF_MASK = 2 ** 16 - 1 # Masking the 16 low bits of an integer
from contextlib import closing

def read_posting_list(inverted, w):
  with closing(MultiFileReader()) as reader:
    locs = inverted.posting_locs[w]
    b = reader.read(locs, inverted.df[w] * TUPLE_SIZE)
    posting_list = []
    for i in range(inverted.df[w]):
      doc_id = int.from_bytes(b[i*TUPLE_SIZE:i*TUPLE_SIZE+4], 'big')
      posting_list.append(doc_id)
    return posting_list

In [153]:
pl = read_posting_list(inverted, 'alabama')
print(pl)

[303, 1285, 1286, 1484]
